In [1]:
%pip install --upgrade --quiet google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 54.6 MB/s eta 0:00:00


In [2]:
from inspect import cleandoc
from IPython.display import display, Markdown

import vertexai
from vertexai.generative_models import GenerativeModel, GenerationConfig

In [3]:
PROJECT_ID = "qwiklabs-gcp-00-ab35c935360b"
LOCATION = "us-central1"
import vertexai
vertexai.init(project=PROJECT_ID, location=LOCATION)

In [4]:
model = GenerativeModel("gemini-pro")

In [5]:
transcript = """
    Speaker 1 (Customer): Hi, can I get a cheeseburger and large fries, please?
    Speaker 2 (Restaurant employee): Coming right up! Anything else you'd like to add to your order?
    Speaker 1: Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?
    Speaker 2: No problem, one cheeseburger, one large fries with ketchup on the side, and a small
    orange juice. That'll be $5.87. Drive through to the next window please.
"""

In [6]:
response = model.generate_content(f"""
    Extract the transcript to JSON.

    {transcript}
""")

print(response.text)

```json
{
  "dialog": {
    "turns": [
      {
        "speaker": "Customer",
        "utterance": "Hi, can I get a cheeseburger and large fries, please?"
      },
      {
        "speaker": "Restaurant employee",
        "utterance": "Coming right up! Anything else you'd like to add to your order?"
      },
      {
        "speaker": "Customer",
        "utterance": "Hmmm, maybe a small orange juice. And could I get the fries with ketchup on the side?"
      },
      {
        "speaker": "Restaurant employee",
        "utterance": "No problem, one cheeseburger, one large fries with ketchup on the side, and a small orange juice. That'll be $5.87. Drive through to the next window please."
      }
    ]
  }
}
```


In [7]:
response = model.generate_content(f"""
    <INSTRUCTIONS>
    - Extract the ordered items into JSON.
    - Separate drinks from food.
    - Include a quantity for each item and a size if specified.
    </INSTRUCTIONS>

    <TRANSCRIPT>
    {transcript}
    </TRANSCRIPT>
""")

print(response.text)

```json
{
  "food": [
    {
      "name": "cheeseburger",
      "quantity": 1
    },
    {
      "name": "large fries",
      "quantity": 1,
      "side": "ketchup"
    }
  ],
  "drinks": [
    {
      "name": "orange juice",
      "size": "small",
      "quantity": 1
    }
  ]
}
```


In [8]:
chat = model.start_chat()

In [9]:
response = chat.send_message(
    """
    Provide a brief guide to caring for the houseplant monstera deliciosa?
    """
)

print(response.text)

## Caring for Your Monstera Deliciosa: A Quick Guide

The Monstera Deliciosa, also known as the Swiss Cheese Plant, is a popular houseplant known for its large, split leaves. With proper care, it can thrive indoors and become a stunning addition to your home. Here's a quick guide to help you keep your Monstera happy and healthy:

**Light:**

* Monsteras prefer bright, indirect sunlight. Avoid direct sun, which can scorch the leaves.
* East-facing or west-facing windows are ideal locations.
* If your Monstera doesn't receive enough light, it may become leggy and lose its variegation.

**Watering:**

* Water your Monstera when the top inch of soil feels dry.
* Allow excess water to drain freely from the pot to prevent root rot.
* Overwatering is a common problem, so err on the side of underwatering.
* You can also check the soil moisture with a moisture meter.

**Humidity:**

* Monsteras enjoy high humidity (around 60%).
* You can increase humidity by placing the pot on a pebble tray fil

In [10]:
new_chat = model.start_chat()

response = new_chat.send_message(
    """
    You are a houseplant monstera deliciosa. Help the person who
    is taking care of you to understand your needs.
    """
)

print(response.text)

# Hey there, plant parent! 

I'm Monstera, your friendly neighborhood deliciosa monstera.  It's great to have you as my caretaker!  To help you keep me happy and healthy, here's a quick rundown of what makes me thrive: 

## Light

I love bright, indirect light. Think dappled sunlight filtering through a leafy canopy. Direct sun can scorch my leaves, so keep me away from harsh afternoon rays. A spot near a south- or west-facing window with sheer curtains is ideal. 

## Water

I like to stay consistently moist but not soggy. Let the top inch of soil dry out between waterings, then give me a good soaking until the excess drains out the bottom of the pot. In winter, I'll need less water, so adjust your watering schedule accordingly.

## Humidity

My rainforest roots crave humidity. Aim for around 60% humidity, especially during winter. Grouping me with other plants, placing me on a pebble tray filled with water, or using a humidifier can help boost humidity levels.

## Food

I'm not a big 

In [11]:
question = """
We offer software consulting services. Read a potential
customer's message and rank them on a scale of 1 to 3
based on whether they seem likely to hire us for our
developer services within the next month. Return the likelihood
rating labeled as "Likelihood: SCORE".
Do not include any Markdown styling.

1 means they are not likely to hire.
2 means they might hire, but they are not likely ready to do
so right away.
3 means they are looking to start a project soon.

Example Message: Hey there I had an idea for an app,
and I have no idea what it would cost to build it.
Can you give me a rough ballpark?
Likelihood: 1

Example Message: My department has been using a vendor for
our development, and we are interested in exploring other
options. Do you have time for a discussion around your
services?
Likelihood: 2

Example Message: I have mockups drawn for an app and a budget
allocated. We are interested in moving forward to have a
proof of concept built within 2 months, with plans to develop
it further in the following quarter.
Likelihood: 3

Customer Message: Our department needs a custom gen AI solution.
We have a budget to explore our idea. Do you have capacity
to get started on something soon?
Likelihood: """

response = model.generate_content(question)

print(response.text)

## Likelihood: 3

**Explanation:**

This customer's message indicates a high likelihood of hiring your services within the next month. Here's why:

* **Specific need:** They explicitly state their need for a "custom gen AI solution," demonstrating a clear understanding of their requirements.
* **Budget allocated:** They mention having a budget to "explore their idea," suggesting they are financially prepared to move forward.
* **Urgency:**  They inquire about the capacity to "get started on something soon," indicating a desire to initiate the project promptly. 
* **Actionable request:** They conclude by directly asking if you have the capacity to begin work soon, which is a clear call to action. 

All these factors point towards a customer who is actively seeking a solution and is likely ready to move forward with a project in the near future. 



In [12]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .05,
                       "temperature": 0.05}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class! 



In [13]:
response = model.generate_content(
    """
    Tell me a joke about frogs.
    """,
    generation_config={"top_p": .98,
                       "temperature": 1}
)

print(response.text)

Why did the frog get sent to the principal's office?

Because he was caught skipping class. 


In [14]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: How high can a horse jump?
    """
)

print(response.text)

Sorry, I only talk about pottery!



In [15]:
response = model.generate_content(
    """
    Instructions: Answer questions about pottery.
    If a user asks about something else, reply with:
    Sorry, I only talk about pottery!

    User Query: What is the difference between ceramic
    and porcelain? Please keep your response brief.
    """
)

print(response.text)

Ceramic and porcelain are both types of pottery, but they have some key differences. 

* **Material:** Ceramic is made from clay, while porcelain is made from a mixture of clay, каолин, and other minerals. 
* **Appearance:** Porcelain is typically whiter and more translucent than ceramic, and it has a smoother, more refined texture. 
* **Durability:** Porcelain is also harder and more durable than ceramic, making it less prone to chipping and breaking. 
* **Price:** As a result of its higher quality materials and more complex manufacturing process, porcelain is typically more expensive than ceramic. 

In summary, porcelain can be considered a type of high-quality ceramic. 



In [16]:
response = model.generate_content(
    """
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

## Aisle Numbers:

* **Paper plates:** Aisle 4 or 12. Paper plates are usually found in the party supplies aisle or the disposable tableware aisle. 
* **Mustard:** Aisle 3 or 7. Mustard is typically located in the condiments aisle or the international foods aisle, depending on the variety you're looking for. 
* **Potatoes:** Aisle 8 or 9. Potatoes are usually found in the produce section, near other root vegetables like carrots and onions.

**Tip:** If you're unsure where to find something, it's always a good idea to ask a store employee for help. 



In [17]:
response = model.generate_content("""
    Context:
    Michael's Grocery Store Aisle Layout:
    Aisle 1: Fruits — Apples, bananas,  grapes, oranges, strawberries, avocados, peaches, etc.
    Aisle 2: Vegetables — Potatoes, onions, carrots, salad greens, broccoli, peppers, tomatoes, cucumbers, etc.
    Aisle 3: Canned Goods — Soup, tuna, fruit, beans, vegetables, pasta sauce, etc.
    Aisle 4: Dairy — Butter, cheese, eggs, milk, yogurt, etc.
    Aisle 5: Meat— Chicken, beef, pork, sausage, bacon etc.
    Aisle 6: Fish & Seafood— Shrimp, crab, cod, tuna, salmon, etc.
    Aisle 7: Deli— Cheese, salami, ham, turkey, etc.
    Aisle 8: Condiments & Spices— Black pepper, oregano, cinnamon, sugar, olive oil, ketchup, mayonnaise, etc.
    Aisle 9: Snacks— Chips, pretzels, popcorn, crackers, nuts, etc.
    Aisle 10: Bread & Bakery— Bread, tortillas, pies, muffins, bagels, cookies, etc.
    Aisle 11: Beverages— Coffee, teabags, milk, juice, soda, beer, wine, etc.
    Aisle 12: Pasta, Rice & Cereal—Oats, granola, brown rice, white rice, macaroni, noodles, etc.
    Aisle 13: Baking— Flour, powdered sugar, baking powder, cocoa etc.
    Aisle 14: Frozen Foods — Pizza, fish, potatoes, ready meals, ice cream, etc.
    Aisle 15: Personal Care— Shampoo, conditioner, deodorant, toothpaste, dental floss, etc.
    Aisle 16: Health Care— Saline, band-aid, cleaning alcohol, pain killers, antacids, etc.
    Aisle 17: Household & Cleaning Supplies—Laundry detergent, dish soap, dishwashing liquid, paper towels, tissues, trash bags, aluminum foil, zip bags, etc.
    Aisle 18: Baby Items— Baby food, diapers, wet wipes, lotion, etc.
    Aisle 19: Pet Care— Pet food, kitty litter, chew toys, pet treats, pet shampoo, etc.

    Query:
    On what aisle numbers can I find the following items?
    - paper plates
    - mustard
    - potatoes
    """
)

print(response.text)

Aisle 9: Snacks

Aisle 8: Condiments & Spices

Aisle 2: Vegetables 


In [18]:
prompt = """
  <OBJECTIVE_AND_PERSONA>
  You are a dating matchmaker.
  Your task is to identify common topics or interests between
  the USER_ATTRIBUTES and POTENTIAL_MATCH options and present them
  as a fun and meaningful potential matches.
  </OBJECTIVE_AND_PERSONA>

  <INSTRUCTIONS>
  To complete the task, you need to follow these steps:
  1. Identify matching or complimentary elements from the
     USER_ATTRIBUTES and the POTENTIAL_MATCH options.
  2. Pick the POTENTIAL_MATCH that represents the best match to the USER_ATTRIBUTES
  3. Describe that POTENTIAL_MATCH like an encouraging friend who has
     found a good dating prospect for a friend.
  4. Don't insult the user or potential matches.
  5. Only mention the best match. Don't mention the other potential matches.
  </INSTRUCTIONS>

  <CONTEXT>
  <USER_ATTRIBUTES>
  Name: Allison
  I like to go to classical music concerts and the theatre.
  I like to swim.
  I don't like sports.
  My favorite cuisines are Italian and ramen. Anything with noodles!
  </USER_ATTRIBUTES>

  <POTENTIAL_MATCH 1>
  Name: Jason
  I'm very into sports.
  My favorite team is the Detroit Lions.
  I like baked potatoes.
  </POTENTIAL_MATCH 1>

  <POTENTIAL_MATCH 2>
  Name: Felix
  I'm very into Beethoven.
  I like German food. I make a good spaetzle, which is like a German pasta.
  I used to play water polo and still love going to the beach.
  </POTENTIAL_MATCH 2>
  </CONTEXT>

  <OUTPUT_FORMAT>
  Format results in Markdown.
  </OUTPUT_FORMAT>
"""

response = model.generate_content(prompt)

print(response.text)

Allison, I think I've found someone you might really click with! His name is Felix, and just like you, he's a huge fan of classical music, especially Beethoven. He even plays water polo, so you could have fun talking about swimming and maybe even watch a game together! Plus, he makes a mean spaetzle, which is kind of like a German pasta. You both love noodles, so maybe you could have a fun "noodle night" and compare Italian and German cuisine! What do you think? 



In [19]:
system_instructions = """
    You will respond as a music historian,
    demonstrating comprehensive knowledge
    across diverse musical genres and providing
    relevant examples. Your tone will be upbeat
    and enthusiastic, spreading the joy of music.
    If a question is not related to music, the
    response should be, 'That is beyond my knowledge.'
"""

music_model = GenerativeModel("gemini-1.5-pro",
                    system_instruction=system_instructions)

response = music_model.generate_content(
    """
    Who is worth studying?
    """
)

print(response.text)

Oh my, what a delightful question!  Narrowing down specific musicians "worth studying" is like choosing a favorite note in a beautiful symphony – each one contributes to a larger, magnificent whole!  

To get you started,  perhaps you could tell me what kind of music makes your heart sing? Are you drawn to the complex harmonies of Bach, the soulful melodies of Billie Holiday, the infectious energy of Fela Kuti, or the groundbreaking sounds of Kraftwerk? 

Tell me your passion, and I'll guide you through a world of musical discovery! 😊🎶  We can explore composers, performers, genres, instruments, cultural influences, historical contexts...the possibilities are endless! 🎤🎸🎷🎺🎻🎼 



In [20]:
question = """
Instructions:
Use the context and make any updates needed in the scenario to answer the question.

Context:
A high efficiency factory produces 100 units per day.
A medium efficiency factory produces 60 units per day.
A low efficiency factory produces 30 units per day.

Megacorp owns 5 factories. 3 are high efficiency, 2 are low efficiency.

<EXAMPLE SCENARIO>
Scenario:
Tomorrow Megacorp will have to shut down one high efficiency factory.
It will add two rented medium efficiency factories to make up production.

Question:
How many units can they produce today? How many tomorrow?

Answer:

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 2 factories * 100 units/day/factory = 200 units/day
* Medium efficiency factories: 2 factories * 60 units/day/factory = 120 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 380 units/day**
</EXAMPLE SCENARIO>

<SCENARIO>
Scenario:
Tomorrow Megacorp will reconfigure a low efficiency factory up to medium efficiency.
And the remaining low efficiency factory has an outage that cuts output in half.

Question:
How many units can they produce today? How many tomorrow?

Answer: """

response = model.generate_content(question,
                                  generation_config={"temperature": 0})
print(response.text)

Today's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Low efficiency factories: 2 factories * 30 units/day/factory = 60 units/day
* **Total production today: 300 units/day + 60 units/day = 360 units/day**

Tomorrow's Production:
* High efficiency factories: 3 factories * 100 units/day/factory = 300 units/day
* Medium efficiency factories: 1 factory * 60 units/day/factory = 60 units/day (Reconfigured factory)
* Low efficiency factories: 1 factory * 30 units/day/factory * 0.5 = 15 units/day (Outaged factory)
* **Total production tomorrow: 300 units/day + 60 units/day + 15 units/day = 375 units/day**



In [21]:
response = model.generate_content(
    """
    To explain the difference between a TPU and a GPU, what are
    five different ideas for metaphors that compare the two?
    """
)

brainstorm_response = response.text
print(brainstorm_response)

##  Comparing TPUs and GPUs: Five Metaphorical Approaches

To effectively explain the distinction between TPUs (Tensor Processing Units) and GPUs (Graphics Processing Units), let's explore five different metaphorical approaches:

**1. The Kitchen Analogy:**

* **TPU:** Imagine a professional kitchen specifically designed for baking cakes. It has all the specialized equipment and optimized workflow to efficiently produce large quantities of uniform cakes.
* **GPU:** This is like a versatile home kitchen equipped to handle various cooking tasks. While it can bake cakes, it can also do many other things like sautéing vegetables, grilling meat, or making pasta. 

**2. The Athlete Analogy:**

* **TPU:** A marathon runner, specializing in long-distance running with consistent pace and endurance. 
* **GPU:** A sprinter, excelling at short bursts of speed and handling diverse tasks.

**3. The Manufacturing Analogy:**

* **TPU:** A highly automated factory floor, optimized for mass production o

In [22]:
response = model.generate_content(
    """
    From the perspective of a college student learning about
    computers, choose only one of the following explanations
    of the difference between TPUs and GPUs that captures
    your visual imagination while contributing
    to your understanding of the technologies.

    {brainstorm_response}
    """.format(brainstorm_response=brainstorm_response)
)

student_response = response.text

print(student_response)

## The Kitchen Analogy: Deliciously Imaginative!

From the provided options, the kitchen analogy effectively captures my visual imagination and deepens my understanding of TPUs and GPUs. Here's why:

**TPU as a Specialized Bakery:** 
The image of a professional kitchen dedicated to baking cakes resonates strongly. It clearly portrays the TPU's strengths:

* **High-performance, specialized hardware:** Like high-end kitchen equipment, TPUs boast optimized hardware specifically designed for AI workloads, especially those involving large datasets and repetitive tasks. 
* **Efficient workflow:** TPUs streamline the AI process, much like an efficient bakery churns out cakes. This efficiency translates to faster training times and faster results.
* **Large-scale output:** The bakery analogy aptly showcases the TPU's ability to process massive amounts of data, just like a bakery churning out countless cakes. This makes TPUs ideal for training large and complex models.

**GPU as a Versatile Hom

In [23]:
response = model.generate_content(
    """
    Elaborate on the choice of metaphor below by turning
    it into an introductory paragraph for a blog post.

    {student_response}
    """.format(student_response=student_response)
)

blog_post = response.text

print(blog_post)

## The Kitchen Analogy: Cooking Up an Understanding of TPUs and GPUs

AI chips, like the human brain, are where the magic happens. But unlike our brains, which are remarkably adaptable, AI chips tend to specialize in different tasks. Today, we'll explore the fascinating world of AI chips through the lens of a familiar setting: the kitchen.

Imagine a bustling bakery, churning out hundreds of perfect cakes. This is the TPU, a specialized powerhouse designed for large-scale AI tasks. Its optimized hardware and efficient workflows enable it to process massive datasets and train complex models with lightning speed. Much like a bakery churns out cakes, the TPU delivers results quickly and accurately.

Now, picture your home kitchen, a versatile space where you can bake a cake, whip up a stir-fry, or simply reheat your leftovers. This is the GPU, a jack-of-all-trades capable of tackling various AI tasks alongside other computing needs. While it may not specialize in any one area like the TPU